In [1]:
import rdflib
from pathlib import Path
import re
import os
import gzip

import polars as pl

In [59]:
df = pl.read_parquet('../data/stats/openaire-lod/openaire-lod.parquet')

In [4]:
df

count,valid,acceptance,collection,line_len,graph_size
i64,bool,str,str,i64,i64
1,true,"""2003-08-07T18:16:49Z""","""2019-05-27T20:23:05Z""",3091,35
2,true,"""2018-01-22""","""2019-02-18T18:14:19Z""",2584,34
3,true,"""2006-05-25T05:52:53Z""","""2019-11-20T21:08:55Z""",3509,41
4,true,"""2007-09-19T13:52:42Z""","""2019-07-29T09:23:07Z""",1354,18
5,true,"""2005-11-02T13:05:16Z""","""2019-11-19T11:17:53Z""",2668,28
…,…,…,…,…,…
29668767,true,"""1995-09-28""","""2020-05-19T18:40:28Z""",2000,24
29668768,true,"""2006-10-27T09:09:27Z""","""2020-06-27T02:10:31Z""",3537,31
29668769,true,"""2009-03-30T16:33:42Z""","""2019-11-22T12:12:20Z""",3700,40


In [60]:
df = df.with_columns(
    pl.col('line_len').shift(1, fill_value=0).cum_sum().alias('offset')
)

In [9]:
df.select(pl.col('valid')).filter(pl.col('valid') == False).count()

valid
u32
9


In [10]:
df.null_count()

count,valid,acceptance,collection,line_len,graph_size
u32,u32,u32,u32,u32,u32
0,0,45025,1187439,0,0


In [11]:
df.max()

count,valid,acceptance,collection,line_len,graph_size
i64,bool,str,str,i64,i64
29668771,true,"""Ă      頓 煥獣愠獵扨敡瀀 ｿｿｿｿ         …","""2020-12-14T16:39:06.786Z""",3387049,8987


In [12]:
df.min()

count,valid,acceptance,collection,line_len,graph_size
i64,bool,str,str,i64,i64
1,false,"""q au ¹i""","""2017-01-31T15:48:12.213Z""",457,0


In [61]:
df_times = df.with_columns(
    pl.col('acceptance').str.replace('Z', '').str.to_datetime(strict=False).alias('acceptance_dt'),
    pl.col('collection').str.to_datetime(strict=False).alias('collection_dt'),
)
df_times

count,valid,acceptance,collection,line_len,graph_size,offset,acceptance_dt,collection_dt
i64,bool,str,str,i64,i64,i64,datetime[μs],"datetime[μs, UTC]"
1,true,"""2003-08-07T18:16:49Z""","""2019-05-27T20:23:05Z""",3091,35,0,2003-08-07 18:16:49,2019-05-27 20:23:05 UTC
2,true,"""2018-01-22""","""2019-02-18T18:14:19Z""",2584,34,3092,2018-01-22 00:00:00,2019-02-18 18:14:19 UTC
3,true,"""2006-05-25T05:52:53Z""","""2019-11-20T21:08:55Z""",3509,41,5677,2006-05-25 05:52:53,2019-11-20 21:08:55 UTC
4,true,"""2007-09-19T13:52:42Z""","""2019-07-29T09:23:07Z""",1354,18,9187,2007-09-19 13:52:42,2019-07-29 09:23:07 UTC
5,true,"""2005-11-02T13:05:16Z""","""2019-11-19T11:17:53Z""",2668,28,10542,2005-11-02 13:05:16,2019-11-19 11:17:53 UTC
…,…,…,…,…,…,…,…,…
29668767,true,"""1995-09-28""","""2020-05-19T18:40:28Z""",2000,24,92231847922,1995-09-28 00:00:00,2020-05-19 18:40:28 UTC
29668768,true,"""2006-10-27T09:09:27Z""","""2020-06-27T02:10:31Z""",3537,31,92231849923,2006-10-27 09:09:27,2020-06-27 02:10:31 UTC
29668769,true,"""2009-03-30T16:33:42Z""","""2019-11-22T12:12:20Z""",3700,40,92231853461,2009-03-30 16:33:42,2019-11-22 12:12:20 UTC


In [62]:
df_times.null_count()

count,valid,acceptance,collection,line_len,graph_size,offset,acceptance_dt,collection_dt
u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,45025,1187439,0,0,0,169417,1187462


In [63]:
df_times.max()

count,valid,acceptance,collection,line_len,graph_size,offset,acceptance_dt,collection_dt
i64,bool,str,str,i64,i64,i64,datetime[μs],"datetime[μs, UTC]"
29668771,true,"""Ă      頓 煥獣愠獵扨敡瀀 ｿｿｿｿ         …","""2020-12-14T16:39:06.786Z""",3387049,8987,92231859098,9193-01-01 00:00:00,2020-12-14 16:39:06.786 UTC


In [64]:
df_times.min()

count,valid,acceptance,collection,line_len,graph_size,offset,acceptance_dt,collection_dt
i64,bool,str,str,i64,i64,i64,datetime[μs],"datetime[μs, UTC]"
1,false,"""q au ¹i""","""2017-01-31T15:48:12.213Z""",457,0,0,0000-01-01 00:00:00,2017-01-31 15:48:12.213 UTC


In [40]:
df_diffs = (
    df_times.select(pl.col('collection_dt').alias('dt'))
    .lazy()
    .drop_nulls()
    .sort('dt')
    .with_columns(pl.col('dt').shift(1).alias('dt_1'))
    .with_columns((pl.col('dt') - pl.col('dt_1')).alias('diff'))
    .drop_nulls()
).collect()
df_diffs

dt,dt_1,diff
"datetime[μs, UTC]","datetime[μs, UTC]",duration[μs]
2017-01-31 15:48:12.334 UTC,2017-01-31 15:48:12.213 UTC,121ms
2017-04-10 14:40:28 UTC,2017-01-31 15:48:12.334 UTC,68d 22h 52m 15s 666ms
2017-05-04 23:04:41 UTC,2017-04-10 14:40:28 UTC,24d 8h 24m 13s
2017-05-04 23:04:41 UTC,2017-05-04 23:04:41 UTC,0µs
2017-05-04 23:04:41 UTC,2017-05-04 23:04:41 UTC,0µs
…,…,…
2020-12-14 16:39:06.717 UTC,2020-12-14 16:39:06.641 UTC,76ms
2020-12-14 16:39:06.718 UTC,2020-12-14 16:39:06.717 UTC,1ms
2020-12-14 16:39:06.782 UTC,2020-12-14 16:39:06.718 UTC,64ms


In [41]:
df_diffs.describe()

statistic,dt,dt_1,diff
str,str,str,str
"""count""","""28481308""","""28481308""","""28481308"""
"""null_count""","""0""","""0""","""0"""
"""mean""","""2019-10-07 16:02:43.979605+00:…","""2019-10-07 16:02:39.693065+00:…","""0:00:04.286539"""
"""std""",null,null,null
"""min""","""2017-01-31 15:48:12.334000+00:…","""2017-01-31 15:48:12.213000+00:…","""0:00:00"""
"""25%""","""2019-04-05 10:40:37+00:00""","""2019-04-05 10:40:37+00:00""","""0:00:00"""
"""50%""","""2019-11-18 23:00:35+00:00""","""2019-11-18 23:00:35+00:00""","""0:00:00"""
"""75%""","""2020-01-09 04:10:44+00:00""","""2020-01-09 04:10:44+00:00""","""0:00:00"""
"""max""","""2020-12-14 16:39:06.786000+00:…","""2020-12-14 16:39:06.785000+00:…","""75 days, 21:35:58"""


In [39]:
(
    df_times.select(pl.col('collection_dt').alias('dt'))
    .lazy()
    .drop_nulls()
    .sort('dt')
).collect()

dt
"datetime[μs, UTC]"
2017-01-31 15:48:12.213 UTC
2017-01-31 15:48:12.334 UTC
2017-04-10 14:40:28 UTC
2017-05-04 23:04:41 UTC
2017-05-04 23:04:41 UTC
…
2020-12-14 16:39:06.717 UTC
2020-12-14 16:39:06.718 UTC
2020-12-14 16:39:06.782 UTC


In [45]:
df_times.drop_nulls('collection_dt').null_count()

count,valid,acceptance,collection,line_len,graph_size,acceptance_dt,collection_dt
u32,u32,u32,u32,u32,u32,u32,u32
0,0,37503,0,0,0,161652,0


In [65]:
df_clean = (
    df_times
    .lazy()
    .drop_nulls('collection_dt')
    .sort('collection_dt', 'count')   
).collect()

In [66]:
df_clean

count,valid,acceptance,collection,line_len,graph_size,offset,acceptance_dt,collection_dt
i64,bool,str,str,i64,i64,i64,datetime[μs],"datetime[μs, UTC]"
25233760,true,null,"""2017-01-31T15:48:12.213Z""",1071,13,78436515451,null,2017-01-31 15:48:12.213 UTC
19771648,true,null,"""2017-01-31T15:48:12.334Z""",1220,16,61457839420,null,2017-01-31 15:48:12.334 UTC
18789661,true,"""2015-07-06""","""2017-04-10T14:40:28Z""",17892,18,58406015260,2015-07-06 00:00:00,2017-04-10 14:40:28 UTC
5150003,true,"""2008-06-30""","""2017-05-04T23:04:41Z""",3678,29,15998328947,2008-06-30 00:00:00,2017-05-04 23:04:41 UTC
18564190,true,"""2008-06-30""","""2017-05-04T23:04:41Z""",4863,32,57700676044,2008-06-30 00:00:00,2017-05-04 23:04:41 UTC
…,…,…,…,…,…,…,…,…
17964052,true,null,"""2020-12-14T16:39:06.717Z""",846,12,55836851750,null,2020-12-14 16:39:06.717 UTC
29247042,true,null,"""2020-12-14T16:39:06.718Z""",922,13,90922039229,null,2020-12-14 16:39:06.718 UTC
23340229,true,"""2015""","""2020-12-14T16:39:06.782Z""",2802,36,72544210204,null,2020-12-14 16:39:06.782 UTC


In [69]:
df_clean.select(pl.col('count'), pl.col('offset'), pl.col('line_len')).write_parquet(
    '../data/stats/openaire-lod/lines-to-include.parquet',
)

In [68]:
df_clean.lazy().select(
    pl.col('line_len'), pl.col('graph_size'),
).describe()

statistic,line_len,graph_size
str,f64,f64
"""count""",2.8481309e7,2.8481309e7
"""null_count""",0.0,0.0
"""mean""",3126.266222,30.459926
"""std""",5796.852205,42.142576
"""min""",602.0,8.0
"""25%""",2324.0,25.0
"""50%""",2978.0,30.0
"""75%""",3627.0,34.0
"""max""",3.387049e6,8987.0


In [54]:
2.8481309e7 * 30.459926

867538564.523134